In [0]:
import numpy as np
import keras
import cv2
from keras import backend as k
from keras.models import Sequential,Model,load_model
from keras.layers import Conv2D, MaxPooling2D, Activation, BatchNormalization, UpSampling2D, Dropout, Flatten, Dense, Input, LeakyReLU, Conv2DTranspose,AveragePooling2D, Concatenate
from keras.layers.merge import concatenate, add
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from tensorflow import set_random_seed
from google.colab import drive
drive.mount('/content/drive')

from PIL import Image
from skimage import color, io
from zipfile import ZipFile 
from io import BytesIO
import matplotlib.pyplot as plt

np.random.seed(1)
set_random_seed(1)

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
class DataGenerator(keras.utils.Sequence):
  'Generates data for Keras'
  
  def __init__(self, list_IDs, zip_file, batch_size=32, dim=(256, 256), n_channels=2, shuffle=True):
    'Initialization'
    self.dim = dim
    self.batch_size = batch_size
    self.list_IDs = list_IDs
    self.zip_file = zip_file
    self.n_channels = n_channels
    self.shuffle = shuffle
    self.on_epoch_end()            # shuffle data on the beginning

  def __len__(self):
    'Denotes the number of batches per epoch'
    return int(np.floor(len(self.list_IDs) / self.batch_size))

  def __getitem__(self, index):
    'Generate one batch of data'
    # Generate indexes of the batch
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

    # Find list of IDs
    list_IDs_temp = [self.list_IDs[k] for k in indexes]

    # Generate data
    X, y = self.__data_generation(list_IDs_temp)

    return X, y

  def on_epoch_end(self):
    'Updates indexes after each epoch'
    self.indexes = np.arange(len(self.list_IDs))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __data_generation(self, list_IDs_temp):
    'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
    # Initialization
    y = np.empty((self.batch_size, *self.dim, self.n_channels), dtype=np.float32)
    X = np.empty((self.batch_size, *self.dim, 1), dtype=np.float32)

    # Generate data
    for i, ID in enumerate(list_IDs_temp):
      img_rgb = np.array(Image.open(BytesIO(self.zip_file.read(ID))).convert('RGB')) / 255
      img_lab = color.rgb2lab(img_rgb)
      l_channel = img_lab[:,:,0]
      l_channel = l_channel / 50 - 1
      l_channel = l_channel[...,np.newaxis]

      ab_channels = img_lab[:,:,1:]
      ab_channels = (ab_channels + 128) / 255 * 2 - 1
      
      y[i,] = ab_channels
      X[i,] = l_channel
            
    return X, y

In [0]:
Input_train=Input((256,256,1))

#encoder part
c1=Conv2D(64,(4,4),padding='same',strides=(1,1),name='conv1')(Input_train)
c1=BatchNormalization()(c1)
c1=LeakyReLU(alpha=0.2)(c1)
c2=Conv2D(64,(4,4),padding='same',strides=(2,2),name='conv2')(c1)
c2=BatchNormalization()(c2)
c2=LeakyReLU(alpha=0.2)(c2)
c3=Conv2D(128,(4,4),padding='same',strides=(2,2),name='conv3')(c2)
c3=BatchNormalization()(c3)
c3=LeakyReLU(alpha=0.2)(c3)
c4=Conv2D(256,(4,4),padding='same',strides=(2,2),name='conv4')(c3)
c4=BatchNormalization()(c4)
c4=LeakyReLU(alpha=0.2)(c4)
c5=Conv2D(512,(4,4),padding='same',strides=(2,2),name='conv5')(c4)
c5=BatchNormalization()(c5)
c5=LeakyReLU(alpha=0.2)(c5)
c6=Conv2D(512,(4,4),padding='same',strides=(2,2),name='conv6')(c5)
c6=BatchNormalization()(c6)
c6=LeakyReLU(alpha=0.2)(c6)
c7=Conv2D(512,(4,4),padding='same',strides=(2,2),name='conv7')(c6)
c7=BatchNormalization()(c7)
c7=LeakyReLU(alpha=0.2)(c7)
c8=Conv2D(512,(4,4),padding='same',strides=(2,2),name='conv8')(c7)
c8=BatchNormalization()(c8)
c8=LeakyReLU(alpha=0.2)(c8)

#decoder part
d1=Conv2DTranspose(512,(4,4),padding='same',strides=(2,2),name='deconv1')(c8)
d1 = concatenate([d1, c7])
d1=BatchNormalization()(d1)
d1=Activation("relu")(d1)
d2=Conv2DTranspose(512,(4,4),padding='same',strides=(2,2),name='deconv2')(d1)
d2 = concatenate([d2, c6])
d2=BatchNormalization()(d2)
d2=Activation("relu")(d2)
d3=Conv2DTranspose(512,(4,4),padding='same',strides=(2,2),name='deconv3')(d2)
d3 = concatenate([d3, c5])
d3=BatchNormalization()(d3)
d3=Activation("relu")(d3)
d4=Conv2DTranspose(256,(4,4),padding='same',strides=(2,2),name='deconv4')(d3)
d4 = concatenate([d4, c4])
d4=BatchNormalization()(d4)
d4=Activation("relu")(d4)
d5=Conv2DTranspose(128,(4,4),padding='same',strides=(2,2),name='deconv5')(d4)
d5 = concatenate([d5, c3])
d5=BatchNormalization()(d5)
d5=Activation("relu")(d5)
d6=Conv2DTranspose(64,(4,4),padding='same',strides=(2,2),name='deconv6')(d5)
d6 = concatenate([d6, c2])
d6=BatchNormalization()(d6)
d6=Activation("relu")(d6)
d7=Conv2DTranspose(64,(4,4),padding='same',strides=(2,2),name='deconv7')(d6)
d7 = concatenate([d7, c1])
d7=BatchNormalization()(d7)
d7=Activation("relu")(d7)
#last layer
last_layer=Conv2D(2,(4,4),padding='same',strides=(1,1),activation='tanh',name='last_layer')(d7)
model = Model(inputs=[Input_train], outputs=[last_layer])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 256, 256, 64) 1088        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 256, 256, 64) 0           batch_normalization_1[0][0]      
_____________________________________

In [0]:
data_path = '/content/drive/My Drive/data_b7b_elcima/colored.zip'
zip_data =  ZipFile(data_path, 'r')
ids = zip_data.namelist()[1:]
np.random.shuffle(ids)
dev_length = int(0.1 * len(ids))
test_length = int(0.05 * len(ids))
test_ids = ids[:test_length]
dev_ids = ids[test_length:dev_length]
train_ids = ids[dev_length:]

In [0]:
epochs = 10
batch_size = 32

training_generator = DataGenerator(train_ids, zip_data, batch_size=batch_size)
validation_generator = DataGenerator(dev_ids, zip_data, batch_size=batch_size)

# compile our model
model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['accuracy'])
checkpoint = ModelCheckpoint(filepath = '/content/drive/My Drive/movieColor_{epoch:02d}-{val_loss:.2f}.hdf5')
callbacks_list = [checkpoint]
# training our model
model.fit_generator(generator=training_generator, validation_data=validation_generator, epochs = epochs, callbacks=callbacks_list, verbose=1)
model.save("/content/drive/My Drive/movieColor.h5")

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1874/1874 [==============================] - 2890s 2s/step - loss: 0.0110 - acc: 0.5302 - val_loss: 0.0081 - val_acc: 0.5358
Epoch 2/10
1874/1874 [==============================] - 2786s 1s/step - loss: 0.0086 - acc: 0.5395 - val_loss: 0.0073 - val_acc: 0.5616
Epoch 3/10
1874/1874 [==============================] - 2789s 1s/step - loss: 0.0078 - acc: 0.5509 - val_loss: 0.0072 - val_acc: 0.5442
Epoch 4/10
 777/1874 [===========>..................] - ETA: 25:56 - loss: 0.0075 - acc: 0.5640

In [0]:
epochs = 10
batch_size = 32

training_generator = DataGenerator(train_ids, zip_data, batch_size=batch_size)
validation_generator = DataGenerator(dev_ids, zip_data, batch_size=batch_size)

model = load_model('/content/drive/My Drive/movieColor_01-0.01.hdf5')
checkpoint = ModelCheckpoint(filepath = '/content/drive/My Drive/movieColor_{epoch:02d}-{val_loss:.2f}.hdf5')
callbacks_list = [checkpoint]
# training our model
model.fit_generator(generator=training_generator, validation_data=validation_generator, epochs = epochs, callbacks=callbacks_list, verbose=1)
model.save("/content/drive/My Drive/movieColor.h5")


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1874/1874 [==============================] - 2710s 1s/step - loss: 0.0050 - acc: 0.6513 - val_loss: 0.0049 - val_acc: 0.6446
Epoch 2/10
1874/1874 [==============================] - 2653s 1s/step - loss: 0.0043 - acc: 0.6728 - val_loss: 0.0050 - val_acc: 0.6351
Epoch 3/10
1874/1874 [==============================] - 2742s 1s/step - loss: 0.0037 - acc: 0.6925 - val_loss: 0.0056 - val_acc: 0.6192
Epoch 4/10
1874/1874 [==============================] - 2667s 1s/step - loss: 0.0033 - acc: 0.7097 - val_loss: 0.0039 - val_acc: 0.6932
Epoch 5/10
1874/1874 [==============================] - 2670s 1s/step - loss: 0.0028 - acc: 0.7284 - val_loss: 0.0038 - val_acc: 0.7003
Epoch 6/10
1874/1874 [==============================] - 2675s 1s/step - loss: 0.0025 - acc: 0.7404 - val_loss: 0.0036 - val_acc: 0.7091
Epoch 7/10
1874/1874 [==============================] - 2685s 1s